In [40]:
import numpy as np

最終更新日:20230206

# 評価指標と目的関数
- 評価指標：モデルの性能を測る指標。真値と予測値から計算できれば制約なない
- 目的関数：モデルの学習で最適化される関数。学習が上手く進むためには微分可能である必要がある。

## カスタム評価指標とカスタム目的変数
- モデル、ライブラリによっては評価指標や目的関数をユーザが定義可能
- 「ユーザが定義した」という意味で「カスタム」という言葉を用いる

## xgboostにおけるカスタム評価指標とカスタム目的関数の実装

In [36]:
import xgboost as xgb
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
import pandas as pd
import os

data_dir = '../../data/titanic'

train_df = pd.read_csv(os.path.join(data_dir, 'train.csv'))
test_df = pd.read_csv(os.path.join(data_dir, 'test.csv'))

feat_keys = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
target_key = ['Survived']

train_df['Sex'] = (train_df['Sex']=='male')
test_df['Sex'] = (train_df['Sex']=='male')

tr_x, va_x, tr_y, va_y = train_test_split(train_df[feat_keys], train_df[target_key])


In [42]:
ft = ['c', 'c', 'q', 'c', 'c', 'q']
dtrain = xgb.DMatrix(tr_x, label=tr_y, feature_types=ft, enable_categorical=True)
dvalid = xgb.DMatrix(va_x, label=va_y, feature_types=ft, enable_categorical=True)

# カスタム目的関数
def logragobj(preds, dtrain):
    labels = dtrain.get_label() # 真のラベルを取得
    preds = 1.0 / (1.0 + np.exp(-preds)) # シグモイド
    grad = preds - labels # 勾配
    hess = preds * (1.0 - preds) # 二回微分値
    return grad, hess

# カスタム評価関数
def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'custom-error', float(sum(labels != (preds > 0.0)) / len(labels))

# ハイパーパラメータ
params = {'silent': 1, 'random_state': 71}
num_round = 50
watchlist =[(dtrain, 'train'), (dvalid, 'eval')]

# モデルの学習
bst = xgb.train(params, dtrain, num_round, watchlist, obj=logragobj, custom_metric=evalerror)

# 予測値の変換
pred_val = bst.predict(dvalid)
pred = 1.0 / (1.0 + np.exp(-pred_val))
logloss = log_loss(va_y, pred)
print(logloss)

[23:54:39] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "silent" } are not used.

[0]	train-rmse:0.37563	train-custom-error:0.18563	eval-rmse:0.41037	eval-custom-error:0.25561
[1]	train-rmse:0.59892	train-custom-error:0.14820	eval-rmse:0.62684	eval-custom-error:0.19282
[2]	train-rmse:0.82042	train-custom-error:0.13174	eval-rmse:0.85013	eval-custom-error:0.21973
[3]	train-rmse:1.03782	train-custom-error:0.12275	eval-rmse:1.06261	eval-custom-error:0.20179
[4]	train-rmse:1.22246	train-custom-error:0.12126	eval-rmse:1.24240	eval-custom-error:0.19282
[5]	train-rmse:1.38677	train-custom-error:0.11527	eval-rmse:1.40381	eval-custom-error:0.18834
[6]	train-rmse:1.53569	train-custom-error:0.11527	eval-rmse:1.54224	eval-custom-error:0.18834
[7]	train-rmse:1.67655	train-custom-error:0.11527	eval-rmse:1.67207	eval-custom-error:0.18386
[8]	train-rmse:1.80314	train-custom-error:0.11377	ev

d:\katsuya\Documents\note\kaggleNote\venv\lib\site-packages\xgboost\core.py:617: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)


[48]	train-rmse:2.97130	train-custom-error:0.05239	eval-rmse:2.92256	eval-custom-error:0.18386
[49]	train-rmse:2.98678	train-custom-error:0.05239	eval-rmse:2.93754	eval-custom-error:0.17937
0.5098499010824544


In [44]:
# 通常の方法で学習を行う場合
params = {'silent':1, 'random_state':71, 'objective':'binary:logistic'}
bst = xgb.train(params, dtrain, num_round, watchlist)

pred = bst.predict(dvalid)
logloss = log_loss(va_y, pred)
print(logloss)

[23:57:25] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "silent" } are not used.

[0]	train-logloss:0.55282	eval-logloss:0.57294
[1]	train-logloss:0.47349	eval-logloss:0.50681
[2]	train-logloss:0.41972	eval-logloss:0.47659
[3]	train-logloss:0.38126	eval-logloss:0.46116
[4]	train-logloss:0.35750	eval-logloss:0.44980
[5]	train-logloss:0.33568	eval-logloss:0.44687
[6]	train-logloss:0.31952	eval-logloss:0.45067
[7]	train-logloss:0.30796	eval-logloss:0.45088
[8]	train-logloss:0.29841	eval-logloss:0.45271
[9]	train-logloss:0.28996	eval-logloss:0.45235
[10]	train-logloss:0.28355	eval-logloss:0.45307
[11]	train-logloss:0.27077	eval-logloss:0.45598
[12]	train-logloss:0.26590	eval-logloss:0.45638
[13]	train-logloss:0.26210	eval-logloss:0.45542
[14]	train-logloss:0.25547	eval-logloss:0.45773
[15]	train-logloss:0.25063	eval-logloss:0.46349
[16]	train-logloss:0.24825	eval-logloss:0.4677

d:\katsuya\Documents\note\kaggleNote\venv\lib\site-packages\xgboost\core.py:617: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)
